In [1]:
import numpy as np
import W2GRASP_GO, W2GRASP_EO
from W2GRASP_GO import write_coord,write_simple_lens
from GO import coor_sys, simple_lens

In [2]:
class simple_lens():
    def __init__(self, coor_sys, 
                 diameter, 
                 refractive_index, loss_tangent = 0, 
                 r1 = None, r2 = None, bs1 = 0, bs2 = 0,
                 thickness = 0,
                 surf_f1 = '',
                 surf_f2 = '',
                 lengthUnit = 'mm',
                 coating_surf1 = '', coating_surf2 = '',
                 name='simple_lens'):
        self.name = name
        self.coor_name = coor_sys
        self.diameter = diameter
        self.refractive_index = refractive_index
        self.loss_tangent = loss_tangent
        self.r1 =r1
        self.r2 = r2
        self.bs1 = bs1
        self.bs2 = bs2
        self.thickness =thickness
        self.surf_f1 = surf_f1
        self.surf_f2 = surf_f2
        self.lengthUnit = lengthUnit
        self.coating_surf1 = coating_surf1
        self.coating_surf2 = coating_surf2

        self.Str = write_simple_lens(self.name,
                                     self.coor_name,
                                     str(self.diameter)+'mm',
                                     self.refractive_index,
                                     self.loss_tangent,
                                     r1 = self.r1, r2 = self.r2, 
                                     bs1 = self.bs1, bs2 = self.bs2,
                                     thickness = self.thickness,
                                     surf1_file = self.surf_f1,surf2_file = self.surf_f2,
                                     lengthUnit_file = self.lengthUnit,
                                     coating_surf1 = self.coating_surf1, coating_surf2 = self.coating_surf2)

In [3]:
coor_ref =coor_sys([0,0,0],[0,0,0],name='coor_feed_ref',)
coor_feed0 =coor_sys([0,0,0],[0,0,0],ref_coor_name = coor_ref.name,name='coor_feed0',)
coor_feed1 =coor_sys([0,0,0],[0,0,0],ref_coor_name = coor_ref.name,name='coor_feed1',)
coor_feed1 =coor_sys([0,0,0],[0,0,0],ref_coor_name = coor_ref.name,name='coor_feed1',)

coor_lens1 =coor_sys([0,0,0],[0,0,0],ref_coor_name = coor_ref.name,name='coor_feed1',)

coor_list=[coor_ref, coor_feed0, coor_feed1, coor_lens1]

In [ ]:
lens1 = simple_lens(coor_feed1.name, 440, 
                 1.525, loss_tangent = 0, 
                 r1 = None, r2 = None, bs1 = 0, bs2 = 0,
                 thickness = 0,
                 surf_f1 = '',
                 surf_f2 = '',
                 lengthUnit = 'mm',
                 coating_surf1 = '', coating_surf2 = '',
                 name='simple_lens')

In [ ]:
for item in coor_list:
    print(item.Str)

In [ ]:
write_simple_lens(name,coord_sys,diameter,
                    refractive_index, loss_tangent=0,
                    r1=None, r2=None,
                    bs1=0,bs2=0,
                    thickness=0,
                    surf1_file='',surf2_file='',lengthUnit_file='mm',
                    coating_surf1='',coating_surf2='')

In [ ]:
coord_Feed_Ref = {'name': 'coor_feed_ref',
                  'origin':[0,0,0],
                  'angle': [0,0,0],
                  'base':'mechanical_axis.cs',
                  'Type':'coor_sys'}

coord_Feed0 = {'name': 'coor_feed',
              'origin': [0,0,0],
              'angle': [0,0,0],
              'base': coord_Feed_Ref['name']}

coord_Feed1 = {'name': 'coor_feed',
              'origin': [100,0,0],
              'angle': [0,0,0],
              'base': coord_Feed_Ref['name']}
